### Task 1 — Quadratic Model Contours & Trust-Region Solutions (Rosenbrock)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def f_rosen(x, a=10.0):
    x = np.asarray(x, dtype=float).reshape(-1)
    return a*(x[1] - x[0]**2)**2 + (1.0 - x[0])**2

def g_rosen(x, a=10.0):
    x1, x2 = float(x[0]), float(x[1])
    g1 = -4.0*a*x1*(x2 - x1**2) - 2.0*(1.0 - x1)
    g2 =  2.0*a*(x2 - x1**2)
    return np.array([g1, g2], dtype=float)

def H_rosen(x, a=10.0):
    x1, x2 = float(x[0]), float(x[1])
    H11 = -4.0*a*(x2 - x1**2) + 8.0*a*x1**2 + 2.0
    H12 = -4.0*a*x1
    H22 =  2.0*a
    return np.array([[H11, H12],[H12, H22]], dtype=float)

In [ ]:
def tr_exact_step(g, B, Delta, tol=1e-10, maxit=100):
    g = np.asarray(g, dtype=float).reshape(-1)
    B = np.asarray(B, dtype=float)
    n = g.size

    w, _ = np.linalg.eigh(B)
    lam_min = float(np.min(w))
    pd = lam_min > 1e-14
    if pd:
        try:
            pB = -np.linalg.solve(B, g)
        except np.linalg.LinAlgError:
            pB = None
        if pB is not None and np.linalg.norm(pB) <= Delta + 1e-14:
            return pB

    lam_lo = max(0.0, -lam_min + 1e-14)
    def p_of_lambda(lam):
        return -np.linalg.solve(B + lam*np.eye(n), g)

    lam = lam_lo * (1.0 + 1e-6) + (1e-8 if lam_lo == 0.0 else 0.0)
    p = p_of_lambda(lam)
    norm_p = np.linalg.norm(p)

    if norm_p < Delta:
        lam = max(0.0, lam_lo * (1.0 - 1e-6) - 1e-12)
        p = p_of_lambda(lam)
        norm_p = np.linalg.norm(p)

    lam_hi = max(1.0, lam)
    p_hi = p_of_lambda(lam_hi)
    norm_hi = np.linalg.norm(p_hi)
    it = 0
    while norm_hi > Delta and it < 60:
        lam_hi *= 2.0
        p_hi = p_of_lambda(lam_hi)
        norm_hi = np.linalg.norm(p_hi)
        it += 1

    lo, hi = lam, lam_hi
    for _ in range(maxit):
        mid = 0.5 * (lo + hi)
        pm = p_of_lambda(mid)
        vm = np.linalg.norm(pm) - Delta
        if abs(vm) <= tol:
            return pm
        if vm > 0:
            lo = mid
        else:
            hi = mid
    return pm

In [ ]:
def plot_model_and_family(xk, a=10.0, deltas=None, grid=2.5, levels=30, figsize=(6,5)):
    if deltas is None:
        deltas = np.linspace(0.2, 2.0, 10)
    f = f_rosen(xk, a=a)
    g = g_rosen(xk, a=a)
    B = H_rosen(xk, a=a)

    P1 = np.linspace(-grid, grid, 401)
    P2 = np.linspace(-grid, grid, 401)
    Z = np.zeros((P1.size, P2.size))
    for i, p1 in enumerate(P1):
        for j, p2 in enumerate(P2):
            p = np.array([p1, p2], dtype=float)
            Z[i, j] = f + g @ p + 0.5 * p @ (B @ p)

    plist = []
    for Delta in deltas:
        pstar = tr_exact_step(g, B, Delta)
        plist.append(pstar)
    P = np.array(plist)

    plt.figure(figsize=figsize)
    plt.contour(P1, P2, Z.T, levels=levels)
    pick = max(1, len(deltas)//5)
    for Delta in deltas[::pick]:
        th = np.linspace(0, 2*np.pi, 300)
        plt.plot(Delta*np.cos(th), Delta*np.sin(th))
    plt.plot(P[:,0], P[:,1], linewidth=1.5)
    plt.scatter(P[:,0], P[:,1], s=10)
    plt.axhline(0, linewidth=0.5); plt.axvline(0, linewidth=0.5)
    plt.title(f"Quadratic model at x=({xk[0]:.3f},{xk[1]:.3f}), a={a}")
    plt.xlabel("p1"); plt.ylabel("p2")
    plt.tight_layout()

In [ ]:
# Generate figures for the two points
plot_model_and_family(np.array([0.0, -1.0]), a=10.0, deltas=np.linspace(0.2, 2.0, 10))
plt.show()

plot_model_and_family(np.array([0.0, 0.5]), a=10.0, deltas=np.linspace(0.2, 2.0, 10))
plt.show()